# True Positive, True Negative, False Negative and False Positive

In classification, the prediction of a model can be generally catogarized into True Positive, True Negative, False Negative and False Positive where

1. True Positive is an outcome where the model correctly predicts the positive class.
2. True Negative is an outcome where the model correctly predicts the negative class.
3. Flase Positive is an outcome where the model incorrectly predicts the positive class.
4. Flase Negative is an outcome where the model incorrectly predicts the negative class.

This can be represented in a matrix as:

|    |    |
|----|----|
| TP | FP |
| FN | TN |

In [3]:
import numpy as np

def get_positives_and_negatives(yTrue, yPredicted):
    """Calculates the True Positive, True Negative, False Negative and False Positive of the given true and predicted numpy arrays.
        Here the arrays are assumed to have only 2 classes.
    
    Args:
        yTrue (numpy ndarray): The true lables. Ground truth.
        yPredict (numpy ndarray): The predicted lables. Predictions.
    
    Raises:
        Assertion Exception: If yTrue or yPredict is not a numpy array
    
    Returns (:obj: numpy ndarray): A numpy array containing True Positive, True Negative, False Negative and False Positive
    
    Example:
        get_positives_and_negatives(np.asarray([0,0,1,0,0,1,0]), np.asarray([0,1,1,0,1,0,0]))
    
    """
    
    if not isinstance(yTrue, np.ndarray):
        raise AssertionError("{} must be of type numpy.ndarray".format(yTrue))
    if not isinstance(yPredicted, np.ndarray):
        raise AssertionError("{} must be of type numpy.ndarray".format(yPredicted))
    
    TP = np.sum(np.logical_and(yPredicted == 1, yTrue == 1))
    
    TN = np.sum(np.logical_and(yPredicted == 0, yTrue == 0))
    
    FP = np.sum(np.logical_and(yPredicted == 1, yTrue == 0))
    
    FN = np.sum(np.logical_and(yPredicted == 0, yTrue == 1))
    
    return np.array([TP, TN, FP, FN])

In [4]:
get_positives_and_negatives(np.asarray([0,0,1,0,0,1,0]), np.asarray([0,1,1,0,1,0,0]))


array([1, 3, 2, 1])

# Confusion matrix

Confusion matrix is used to visualize the performance of a classification model. In this matrix, the rows are the predicted values and the columns are the true values. The name stems from the fact that it makes it easy to see if the model is confusing two classes.

|        | Cat | Dog | Rabbit |
|--------|-----|-----|--------|
| Cat    | 5   | 2   | 0      |
| Dog    | 3   | 3   | 0      |
| Rabbit | 1   | 1   | 5      |

In [48]:
import numpy as np
from scipy.sparse import coo_matrix

def confusion_matrix(y_pred, y_true):
    """Builds a confusion matrix for the given predicted and trye numpy array.
    
    Args:
        y_pred (numpy ndarray): The true lables. Ground truth.
        y_true (numpy ndarray): The predicted lables. Predictions.
    
    Raises:
        Assertion Exception: If yTrue or yPredict is not a numpy array
    
    Returns:
        C (numpy ndarray): The confusion matrix that was calculated.
    
    Example:
        y_true = [2, 0, 2, 2, 0, 1]
        y_pred = [0, 0, 2, 2, 0, 2]
        confusion_matrix(y_pred, y_True)
    
    """
    if not isinstance(y_true, np.ndarray):
        raise AssertionError("{} must be of type numpy.ndarray".format(y_true))
    
    if not isinstance(y_pred, np.ndarray):
        raise AssertionError("{} must be of type numpy.ndarray".format(y_pred))
    
    if not (y_pred.size == y_true.size):
        raise AssertionError("The predicted array and the ground truth arrays must be of the same size")
    
    _labelsT = np.unique(y_true) #Get the lables in the ground truth.
    
    labels = _labelsT #Create a array of lables.
    
    weights = np.ones(y_true.shape[0]) # Create an array of ones so that we can pass this as the basic building block to confusion matrix. 
    
    num_labels = labels.size
    
    #Builds a dict that converts lables to indices. This will help us populate the confusion matrix.
    
    labels_to_index = dict((label, index) for index, label in enumerate(labels)) 
    
    #Build y_pred_idx and y_true_idx numpy arrays that has the index equivalent of the lables
    
    y_pred_idx = np.array([labels_to_index.get(label, num_labels+1) for label in y_pred])
    
    y_true_idx = np.array([labels_to_index.get(label, num_labels+1) for label in y_true])
    
    ind = np.logical_and(y_pred_idx < num_labels, y_true_idx < num_labels)
    
    y_pred_idx = y_pred_idx[ind]
    y_true_idx = y_true_idx[ind]
    weights = weights[ind]
    
    CM = coo_matrix((weights, (y_true_idx, y_pred_idx)), shape = (num_labels, num_labels)).toarray()
    return CM

In [49]:
y_true = np.array([2, 0, 2, 2, 0, 1])
y_pred = np.array([0, 0, 2, 2, 0, 2])
confusion_matrix(y_pred, y_true)

array([[2., 0., 0.],
       [0., 0., 1.],
       [1., 0., 2.]])

In [38]:
confusion_matrix(y_pred, y_true).ravel()

array([2, 0, 0, 0, 0, 1, 1, 0, 2], dtype=int8)

In [36]:
confusion_matrix(np.array([1, 1, 1, 0]), np.array([0, 1, 0, 1]))

array([[0, 2],
       [1, 1]], dtype=int8)

In [37]:
tn, fp, fn, tp = confusion_matrix(np.array([1, 1, 1, 0]), np.array([0, 1, 0, 1])).ravel()
(tn, fp, fn, tp)

(0, 2, 1, 1)

# Accuracy

This is the fraction of the predictioins that were predicted right. It is given by:

$$\frac{True Positive + True Negative}{True Positive + True Negative + False Negative + False positive}$$

When working with class imbalence data i.e. where the number of examples of each class is different, accuracy doesn't always give the full story. Here the model has been trained on one class more than on another and the test data also has an imbalence thus resulting in higer accuracy. 

# Precision and Recall

Precision is the fraction of the predicted values that were true positive i.e. are actually correct.

It is defined as 
$$\frac{True Positive}{True Positive + False Positive}$$

Recall is the fraction of the predicteds that were identified correctly.

It is defined as:

$$\frac{True Positive}{True Positive + False Negative}$$

**In simple terms, high precision means that an algorithm returned substantially more relevant results than irrelevant ones, while high recall means that an algorithm returned most of the relevant results.**

# F1 Score

F1 score is a measure of a test's accuracy. It is the harmonic mean of Precision and Recall.

It is defined as:

$$2 \times \frac{Precision \times Recall}{Precision + Recall}$$

In [64]:
def get_precision_recall_f1score(y_pred, y_true):
    """Calculates precision, recall and f1score the given predicted and trye numpy array.
    
    Args:
         yTrue (numpy ndarray): The true lables. Ground truth.
        yPredict (numpy ndarray): The predicted lables. Predictions.
    
    Raises:
        Assertion Exception: If yTrue or yPredict is not a numpy array
    
    Returns:
        cr (dictionary of precision, recall and f1score for each of the label): The precision, recall and f1score for each of the label.
    
    Example:
        y_true = [0, 1, 2, 2, 2]
        y_pred = [[0, 0, 2, 2, 1]
        get_precision_recall_f1score(y_pred, y_True)
    
    """
    CM = confusion_matrix(y_pred, y_true)
    
    labels = np.unique(y_true)
    
    cr = dict()
    
    for label in labels:
        tp = CM[label, label]
        fp = CM[:,label].sum() - tp
        fn = CM[label,:].sum() -tp
        prec = np.around(tp/(tp+fp), decimals = 2)
        recall = np.around(tp/(tp+fn), decimals = 2)
        if prec == 0 and recall == 0 :
            f1 = 0.0
        else:
            f1 = np.around(2* (prec * recall)/(prec + recall), decimals = 2)
        cr[label] = (prec, recall, f1)
    
    return cr
    

In [65]:
y_true = np.array([0, 1, 2, 2, 2])
y_pred = np.array([0, 0, 2, 2, 1])
get_precision_recall_f1score(y_pred, y_true)

{0: (0.5, 1.0, 0.67), 1: (0.0, 0.0, 0.0), 2: (1.0, 0.67, 0.8)}